# Imports

In [2]:
import os
import pandas as pd
import numpy as np
import itertools

# Obtain the XT Data

In [4]:
# Designate the path to the directory containing all XT Page code files
path_XTP = "C:\\Users\\sdean\\Desktop\\Sierra Code\\XTP_Directory\\"
files_XTP = [file for file in os.listdir(path_XTP)]

In [5]:
# Create an empty dataframe to be used for information from all XT Page source files
All_XTP = pd.DataFrame(dtype=object)
# Perform this looped operation for each file in the directory
for file in files_XTP:
    # Read the XT Page code files
    d = pd.read_csv(path_XTP+file, header=None)
    df = pd.DataFrame(data=d, dtype=object)
    df = df.rename(columns={df.columns[0]:"Text"})
    # Remove all "Internal Information" markers, which disrupt the information capture
    bad_index = ["internalRead", "internal read", "INternal Read", "--", "param", "cmcmem_word"]
    for bad in bad_index:
        ind = df[df["Text"].str.contains(bad)].index
        df = df.drop(index=ind, axis=0)
        df = df.reset_index(drop=True)
    # Create the Thumbwheel and XT Page dataframe columns
    df["Thumbwheel"] = "N/A"
    df["XT Page"] = file
    # Remove all rows which arent a Thumbwheel name or designation
    for i in df["Text"].index:
        if "//" not in df["Text"][i]:
            if "ccs" not in df["Text"][i]:
                df = df.drop(index=i, axis=0)
    df = df.reset_index(drop=True)
    # Move the Thumbwheel designations to their own column
    for j in df["Text"].index:
        if "ccs" in df["Text"][j]:
            df.loc[j-1, "Thumbwheel"] = df["Text"][j]
    # Remove all rows which don't have a working Thumbwheel designation
    for k in df["Text"].index:
        if "N/A" in df["Thumbwheel"][k]:
            df = df.drop(index=k, axis=0)
    # Append the dataframe created for this file to a dataframe of all files in the directory         
    All_XTP = pd.concat([All_XTP, df], axis=0)
    All_XTP = All_XTP.reset_index(drop=True)
# Format the dataframe
All_XTP["Text"] = All_XTP["Text"].str.upper()
All_XTP["Text"] = All_XTP["Text"].str.replace("//", "")
All_XTP["Text"] = All_XTP["Text"].str.replace("\t", "")
All_XTP["Text"] = All_XTP["Text"].str.strip()
All_XTP["Thumbwheel"] = All_XTP["Thumbwheel"].str.replace("ccs", "")
All_XTP["Thumbwheel"] = All_XTP["Thumbwheel"].str.replace("\t", "")
All_XTP["Thumbwheel"] = All_XTP["Thumbwheel"].str.replace('"', "")
All_XTP["Thumbwheel"] = All_XTP["Thumbwheel"].str.strip()
All_XTP["XT Page"] = All_XTP["XT Page"].str.replace(".txt", "")
All_XTP["XT Page"] = All_XTP["XT Page"].str.replace("XTP_", " ")
All_XTP["XT Page"] = All_XTP["XT Page"].str.strip()
# Designate the naming convention for the Thumbwheel systems
Key_TW = ["B1", "V1", "2A", "2V", "B2", "B4", "V4", "CD", "DG", "ET", "I1", "I2", "PV", "I3", "CV", "I4", "EV", "LS", "IB", "IV", "MG", 
        "MS", "PT", "RF", "SB", "SF", "TG", "TK", "TV"]
Key_Search = ["Beamline 1", "Beamline 1 Vacuum", "Beamline 2A", "Beamline 2A Vacuum", "Beamline 2C", "Beamline 4", "Beamline 4 Vacuum", 
              "CAMAC Design", "Diagnostics", "Extra Thumbwheels", "Ion Source 1", "Ion Source 2", "Ion Source 2 Vacuum", "Ion Source 3", 
              "Ion Source 3 Vacuum", "Ion Source 4", "Ion Source 4 Vacuum", "Ion Source 4 Laser", "ISIS Beamline", "ISIS Beamline Vacuum", 
              "Magnets", "Miscellaneous", "Proton Therapy", "Radio Frequency", "Secondary Beamlines", "Safety", "Targets", "Tank", "Tank Vacuum"]
# Remove all Thumbwheels that are not in the systems
Bad_df = All_XTP
for l in range(0, len(Key_TW)):
    indexes = Bad_df[Bad_df["Thumbwheel"].str.contains(Key_TW[l])].index
    Bad_df = Bad_df.drop(indexes, axis=0)
All_XTP = All_XTP.drop(Bad_df.index, axis=0)
All_XTP = All_XTP.reset_index(drop=True)
# Create a column in the dataframe for the Thumbwheel systems
All_XTP["System"] = "N/A"
for m in range(0, len(Key_TW)):
    Key_Indexes = All_XTP[All_XTP["Thumbwheel"].str.contains(Key_TW[m])].index
    All_XTP.loc[Key_Indexes, "System"] = Key_Search[m]
# Identify each unique instance of repeated thumbwheels found on multiple pages
Drop_All = []
Duplicates = All_XTP[(All_XTP["Thumbwheel"].duplicated() == True)]
Duplicate_TWs = list(np.unique(Duplicates["Thumbwheel"]))
# Group together all the thumbwheel information from each page
for n in Duplicate_TWs:
    indexes = []
    for o in range(0, len(All_XTP["Thumbwheel"])):
        if All_XTP["Thumbwheel"][o] == n:
            indexes.append(o)
    Duplicate_Titles = str(list(np.unique(All_XTP["Text"].loc[indexes])))
    Duplicate_XTPs = str(list(np.unique(All_XTP["XT Page"].loc[indexes])))
    All_XTP.loc[indexes[0], "Text"] = Duplicate_Titles
    All_XTP.loc[indexes[0], "XT Page"] = Duplicate_XTPs
    a = indexes.pop(0)
    for p in indexes:
        Drop_All.append(p)
# Removed all duplicated instances so each thumbwheel only appears in the dataframe once
All_XTP = All_XTP.drop(Drop_All, axis=0)
All_XTP = All_XTP.reset_index(drop=True)
# Remove thumbwheels no longer in use
indexe = All_XTP[All_XTP["Thumbwheel"].str.contains("//")].index
All_XTP = All_XTP.drop(index=indexe, axis=0)
All_XTP = All_XTP.reset_index(drop=True)
# Format the dataframe
All_XTP["Text"] = All_XTP["Text"].str.replace("[", "")
All_XTP["Text"] = All_XTP["Text"].str.replace("]", "")
All_XTP["Text"] = All_XTP["Text"].str.replace("'", "")
All_XTP["XT Page"] = All_XTP["XT Page"].str.replace("[", "")
All_XTP["XT Page"] = All_XTP["XT Page"].str.replace("]", "")
All_XTP["XT Page"] = All_XTP["XT Page"].str.replace("'", "")
# This results in a dataframe containing all active thumbwheels, their associated text,
# their associated system, and the XT Pages they exist on.

In [6]:
# print(All_XTP)

# Obtain the Scan Data

In [8]:
# Designate the path to the directory containing all Scan source files
path_Scan = "C:\\Users\\sdean\\Desktop\\Sierra Code\\Scan_Directory\\"
files_Scan = [file for file in os.listdir(path_Scan)]

In [9]:
# Create an empty dataframe to be used for information from all scans
All_Scans = pd.DataFrame(columns = ["Scan", "Element", "On Action", "DO OFF Command", "DO MESSAGE Command", "DO WRITE Command", 
                                    "DO INSERT Command", "DO CALC Command", "DO LOG Command", "DO CLEAR Command"], dtype=object)
# Perform this looped operation for each file in the directory
for file in files_Scan:
    # Create an empty dataframe to be re-used for each file 
    df_scan = pd.DataFrame(index=[0], columns = ["Scan", "Element", "On Action", "DO OFF Command", "DO MESSAGE Command", "DO WRITE Command", 
                                             "DO INSERT Command", "DO CALC Command", "DO LOG Command", "DO CLEAR Command"], dtype=object)
    # Read the Scan source files
    ds = pd.read_csv(path_Scan+file, header=None)
    dfs = pd.DataFrame(data=ds, dtype=str)
    dfs = dfs.rename(columns={dfs.columns[0]:"Scan"})
    # Remove all "Internal Information" markers, which disrupt the information capture 
    comments = dfs[dfs["Scan"].str.contains("//")].index
    dfs = dfs.drop(comments, axis=0)
    dfs = dfs.reset_index(drop=True)
    freq = dfs[dfs["Scan"].str.contains("FREQUENCY:")].index
    dfs = dfs.drop(freq, axis=0)
    dfs = dfs.reset_index(drop=True)
    peri = dfs[dfs["Scan"].str.contains("PERIOD:")].index
    dfs = dfs.drop(peri, axis=0)
    dfs = dfs.reset_index(drop=True)
    numel = dfs[dfs["Scan"].str.contains("NUMBER OF ELEMENTS:")].index
    dfs = dfs.drop(numel, axis=0)
    dfs = dfs.reset_index(drop=True)
    disac = dfs[dfs["Scan"].str.contains("ON DISABLED ACTIONS:")].index
    dfs = dfs.drop(disac, axis=0)
    dfs = dfs.reset_index(drop=True)
    trueac = dfs[dfs["Scan"].str.contains("ON TRUE ACTIONS:")].index
    dfs = dfs.drop(trueac, axis=0)
    dfs = dfs.reset_index(drop=True)
    # Separate the code blocks into the scan element and relevant commands
    count = -1
    for q in dfs["Scan"].index:
        if "ELEMENT" in dfs["Scan"][q]:
            count += 1
            df_scan.loc[count, "Element"] = dfs["Scan"][q]
        elif "DO OFF" in dfs["Scan"][q]:
            if pd.isna(df_scan.at[count, "DO OFF Command"]) == False:
                df_scan.loc[count, "DO OFF Command"] = df_scan.loc[count, "DO OFF Command"] + ", " + dfs["Scan"][q]
            else:
                df_scan.loc[count, "DO OFF Command"] = dfs["Scan"][q]
        elif "DO MESSAGE" in dfs["Scan"][q]:
            df_scan.loc[count, "DO MESSAGE Command"] = dfs["Scan"][q]
        elif "DO WRITE" in dfs["Scan"][q]:
            if pd.isna(df_scan.at[count, "DO WRITE Command"]) == False:
                df_scan.loc[count, "DO WRITE Command"] = df_scan.loc[count, "DO WRITE Command"] + ", " + dfs["Scan"][q]
            else:
                df_scan.loc[count, "DO WRITE Command"] = dfs["Scan"][q]
        elif "DO INSERT" in dfs["Scan"][q]:
            df_scan.loc[count, "DO INSERT Command"] = dfs["Scan"][q]
        elif "DO CALC" in dfs["Scan"][q]:
            if pd.isna(df_scan.at[count, "DO CALC Command"]) == False:
                df_scan.loc[count, "DO CALC Command"] = df_scan.loc[count, "DO CALC Command"] + ", " + dfs["Scan"][q]
            else:
                df_scan.loc[count, "DO CALC Command"] = dfs["Scan"][q]
        elif "DO LOG" in dfs["Scan"][q]:
            df_scan.loc[count, "DO LOG Command"] = dfs["Scan"][q]
        elif "DO CLEAR" in dfs["Scan"][q]:
            df_scan.loc[count, "DO CLEAR Command"] = dfs["Scan"][q]
        else:
            if pd.isna(df_scan.at[count, "On Action"]) == False:
                df_scan.loc[count, "On Action"] = df_scan.loc[count, "On Action"] + ", " + dfs["Scan"][q]
            else:
                df_scan.loc[count, "On Action"] = dfs["Scan"][q]
    # Append the dataframe created for this file to a dataframe of all files in the directory 
    df_scan["Scan"] = file
    All_Scans = pd.concat([All_Scans, df_scan], axis=0)
    All_Scans = All_Scans.reset_index(drop=True)
# Removed all the Elements that are disabled
disab = All_Scans[All_Scans["Element"].str.contains('DISABLED')].index
All_Scans = All_Scans.drop(disab, axis=0)
All_Scans = All_Scans.reset_index(drop=True)
for r in All_Scans["Element"].index:
    All_Scans.loc[r, "Element"] = All_Scans["Element"][r].split(":")[0]
All_Scans["Element"] = All_Scans["Element"].str.replace("ELEMENT ", "")
# Format the dataframe
All_Scans["Scan"] = All_Scans["Scan"].str.replace("_Scan.csv", "")
All_Scans = All_Scans.fillna('N/A')
# Format the Thumbwheels from SYS,NUM,ID to SYS NUM ID, as it is printed in the other files
for s in All_Scans.columns:
    All_Scans[s] = All_Scans[s].str.replace(", ", "!!!") 
    All_Scans[s] = All_Scans[s].str.replace(",", " ")
    All_Scans[s] = All_Scans[s].str.replace("!!!", ", ") 
# Format the Message Commands to remove symbols 
All_Scans["DO MESSAGE Command"] = All_Scans["DO MESSAGE Command"].str.replace("mainlog ", "")
All_Scans["DO MESSAGE Command"] = All_Scans["DO MESSAGE Command"].str.replace("opslog ", "") 
All_Scans["DO MESSAGE"] = All_Scans["DO MESSAGE Command"]
All_Scans["DO MESSAGE Command"] = All_Scans["DO MESSAGE Command"].str.strip()
symb = All_Scans[All_Scans["DO MESSAGE Command"].str.contains('"')].index
for t in symb:
    All_Scans["DO MESSAGE Command"][t] = All_Scans["DO MESSAGE Command"][t].split('"')[0]
# This results in a dataframe containing all scans, their associated elements, the initial
# action which promts the scan response, and all resulting actions.

In [10]:
# print(All_Scans)

# Associate Thumbwheels with Scan Elements
Cell must be run twice to overcome the error; Known issue, does not affect outcome.

In [16]:
# Determine the thumbwheels in the scan data
for u in range(0, len(All_XTP["Thumbwheel"])):
    ind = [All_Scans[All_Scans[col].str.contains(All_XTP["Thumbwheel"][u])].index for col in All_Scans]
    merged_ind = list(itertools.chain.from_iterable(ind))
    # Record the scan and the element which contain each Thumbwheel
    scan = [] 
    element = []
    if len(merged_ind) != 0:
        count = 0
        for v in merged_ind:
            if All_Scans["Scan"][v] not in scan:
                scan.append(All_Scans["Scan"][v])
                element.insert(count, [str(All_Scans["Scan"][v])]) 
                element[count].append(int(All_Scans["Element"][v])) 
                count += 1
            else:
                element[count-1].append(int(All_Scans["Element"][v]))
    # Append the scan and element information to the dataframe for all XT data 
    if len(scan) != 0:        
        All_XTP.at[u, "Scan"] = scan
        All_XTP.at[u, "Scan Element"] = element
All_XTP["Scan"] = All_XTP["Scan"].apply(lambda d: d if isinstance(d, list) else "N/A")
All_XTP["Scan Element"] = All_XTP["Scan Element"].apply(lambda d: d if isinstance(d, list) else "N/A")
# This adds to the dataframe containing all active thumbwheels, their associated text,
# their associated system, and the XT Pages they exist on, adding their associated scans 
# and scan elements

In [17]:
# print(All_XTP)

# Obtain the Interlock Statement Data

In [19]:
# Designate the path to the directory containing all Interlock source files
path_I = "C:\\Users\\sdean\\Desktop\\Sierra Code\\Scan_ILStat\\"
files_I = [file for file in os.listdir(path_I)]

In [20]:
# Create an empty dataframe to be used for the interlock statements from all files
All_IL = pd.DataFrame(columns = ["Scan", "ELT #", "I/L Statement", "Message log message printed"], dtype=str)
# Perform this looped operation for each file in the directory
for file in files_I:
    # Read the Interlock source files
    di = pd.read_csv(path_I+file, skiprows=4, dtype=str)
    # Create an empty dataframe to be re-used for each file 
    ddi = pd.DataFrame(data=di, columns=(["ELT #", "I/L Statement", "Message log message printed"]))
    ddi = ddi.dropna(how="all")
    ddi = ddi.reset_index(drop=True)
    ddi["Scan"] = file
    # Append the dataframe created for this file to a dataframe of all files in the directory 
    All_IL = pd.concat([All_IL, ddi])
    All_IL = All_IL.reset_index(drop=True)
# Format the dataframe
All_IL["Scan"] = All_IL["Scan"].str.replace(".csv", "")
All_IL = All_IL.rename(columns={"ELT #": "Element", "Message log message printed":"CCS Message"}, inplace=False)
# Remove any disabled messages
All_IL = All_IL.fillna('N/A')
dis = All_IL[All_IL["I/L Statement"].str.contains("Disabled")].index
All_IL = All_IL.drop(dis, axis=0)
# Format the dataframe
All_IL["Scan"] = All_IL["Scan"].str.replace("_MSG", "")
All_IL = All_IL.reset_index(drop=True)
# This results in a dataframe containing all scans, scan elements,
# their associated interlock statements, and the resulting 
# CCS message printed.

In [21]:
# print(All_IL)

# Associate Interlock Statements with Scans

In [23]:
# Search through each scan element with an associated interlock statement
for w in All_IL.index:
    IL_Scan = All_IL["Scan"][w]
    IL_Element = All_IL["Element"][w]
    IL_Interlock = All_IL["I/L Statement"][w]
    IL_CCS = All_IL["CCS Message"][w]
    # Append the interlock and ccs message to the original scan dataframe
    Both_Scans = All_Scans[All_Scans["Scan"]==IL_Scan].index
    Both_Elements = All_Scans[All_Scans["Element"]==IL_Element].index
    common = list(set(Both_Scans) & set(Both_Elements))
    if len(common) != 0:
        All_Scans.loc[common[0], "I/L Statement"] = IL_Interlock
        All_Scans.loc[common[0], "CCS Message"] = IL_CCS
# Format the dataframe
All_Scans = All_Scans.fillna('N/A')
# This adds to the dataframe containing all scans, their associated elements, 
# the initial action which promts the scan response, and all resulting actions, 
# adding their associated interlock statement and CCS messages

In [24]:
# print(All_Scans)

# Obtain the Messages Data

In [26]:
# Designate the path to the directory containing the Message file
path_M = "C:\\Users\\sdean\\Desktop\\Sierra Code\\MES2.csv"
# Read the Messages source file
dm = pd.read_csv(path_M, dtype=str, header=None)
# Create an dataframe containing the Message number and associated message
ddm = pd.DataFrame(data=dm)
ddm = ddm.rename(columns={0: "First"})
ddm[['Message Number', 'Message']] = ddm['First'].str.split(' ', n=1, expand=True)
ddm = ddm.drop(['First'], axis=1)
ddm['Message Number'] = ddm['Message Number'].str.strip()
ddm['Message'] = ddm['Message'].str.strip()
# This results in a dataframe containing Message numbers and their
# associated Message.

In [27]:
# print(ddm)

# Associate Output Message with Scan DO MESSAGE Command

In [29]:
# Determine all Scans which contain a Message Number
Mes_list = All_Scans["DO MESSAGE Command"]
Nas = All_Scans[All_Scans["DO MESSAGE Command"]=="N/A"].index
# Create a list of all Message numbers which need an associated Message
Mes_list = Mes_list.drop(index=Nas)
Mes_list = Mes_list.str.replace("DO MESSAGE", "")
Mes_list = Mes_list.str.strip()
# Append the Message Number and Message to the original scan dataframe
for x in range(0, len(Mes_list.index)):
    Place = ddm[ddm["Message Number"]==(Mes_list[Mes_list.index[x]])].index
    a = ddm["Message Number"][Place[0]]
    b = All_Scans["DO MESSAGE"][Mes_list.index[x]]
    b = b.replace("DO MESSAGE "+a, "")
    c = ddm["Message"][Place[0]]
    All_Scans.loc[Mes_list.index[x], "DO MESSAGE"] = "DO MESSAGE "+a+": "+c+b
# Format the dataframe
All_Scans["DO MESSAGE"] = All_Scans["DO MESSAGE"].str.replace('"', "")
# This adds to the dataframe containing all scans, their associated elements, 
# the initial action which promts the scan response, and all resulting actions, 
# adding the full message printed when a scan action is triggered.

In [30]:
# print(All_Scans)

# Save the Resulting Data

In [32]:
# Remove the ANSI color code before saving the data
All_Scans["DO MESSAGE"] = All_Scans["DO MESSAGE"].str.replace("\033[31m", "") 
All_Scans["DO MESSAGE"] = All_Scans["DO MESSAGE"].str.replace("\033[33m", "") 
All_Scans["DO MESSAGE"] = All_Scans["DO MESSAGE"].str.replace("[1m[33m", "") 

# Save the data
# All_XTP.to_csv('XTP_2025.csv', index=False)
# All_Scans.to_csv('SCNS_2025.csv', index=False)